In [2]:
import spkit as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from copy import deepcopy

In [3]:
raw = np.array(pd.read_csv("K:\Clean Data\IGE\shirband\LB.csv", skiprows = 2))

In [4]:
X = deepcopy(raw[1080000:1090000, 2:])

In [5]:
ch_names = [str(i) for i in range(X.shape[1])]
fs = 300